In [12]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW


In [14]:
# ------------------------------------------------------------------
# Directory Setup for Saving the Model and Tokenizer
# ------------------------------------------------------------------
SAVE_DIR = "saved_model_fakenewsnet_roberta_title_smote"
os.makedirs(SAVE_DIR, exist_ok=True)

# ------------------------------------------------------------------
# Step 1: Load Data using Only "title"
# ------------------------------------------------------------------
# Replace these paths with your actual CSV file locations.
fake_news = pd.read_csv('data/fake_with_metadata.csv')
real_news = pd.read_csv('data/real_with_metadata.csv')

# Work only with the title column; assign labels (1 for Fake, 0 for Real)
fake_news = fake_news[['title']].copy()
real_news = real_news[['title']].copy()
fake_news['labels'] = 1
real_news['labels'] = 0

# Combine the datasets and drop any records missing title
data = pd.concat([fake_news, real_news], ignore_index=True)
data.dropna(subset=['title'], inplace=True)

# Shuffle the data
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# ------------------------------------------------------------------
# Step 2: Split Data into Training and Testing Sets
# ------------------------------------------------------------------
# We stratify on 'labels' to preserve class proportions.
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, stratify=data['labels'])
print("Number of training samples:", len(train_df))
print("Number of testing samples:", len(test_df))

Number of training samples: 18556
Number of testing samples: 4640


In [15]:
# ------------------------------------------------------------------
# Step 3: Apply SMOTE Using TF-IDF Features on Training Titles
# ------------------------------------------------------------------
# First, vectorize the training titles using TF-IDF.
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(train_df['title']).toarray()
y_train = train_df['labels'].values

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Define a helper function to convert numeric TF-IDF rows back into text.
def tfidf_to_text(X):
    texts = []
    for row in X:
        # inverse_transform returns a list of tokens that had nonzero TF-IDF values.
        tokens = vectorizer.inverse_transform(row.reshape(1, -1))[0]
        texts.append(" ".join(tokens))
    return texts

# Convert the SMOTE-resampled training features back into text strings.
train_texts = tfidf_to_text(X_train_resampled)

# For the test set, just transform using the vectorizer (without SMOTE) and inverse transform.
X_test_tfidf = vectorizer.transform(test_df['title']).toarray()
test_texts = tfidf_to_text(X_test_tfidf)

# Convert labels to tensors
train_labels = torch.tensor(y_train_resampled, dtype=torch.long)
test_labels = torch.tensor(test_df['labels'].values, dtype=torch.long)

C:\Users\Koon\anaconda3\envs\GPU\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [16]:
# ------------------------------------------------------------------
# Step 4: Tokenize the Titles Using RoBERTa Tokenizer
# ------------------------------------------------------------------
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_texts(texts, max_len=128):
    return tokenizer(texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt')

train_encodings = tokenize_texts(train_texts)
test_encodings = tokenize_texts(test_texts)

# ------------------------------------------------------------------
# Step 5: Create a Custom Dataset Class
# ------------------------------------------------------------------
class TitleDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = TitleDataset(train_encodings, train_labels)
test_dataset = TitleDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(test_dataset, batch_size=16)

In [17]:
# ------------------------------------------------------------------
# Step 6: Load and Adapt the RoBERTa Model for Sequence Classification
# ------------------------------------------------------------------
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 5  # Using 5 epochs

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Koon\anaconda3\envs\GPU\lib\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
# ------------------------------------------------------------------
# Step 7: Train the Model
# ------------------------------------------------------------------
print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch in loop:
        optimizer.zero_grad()
        # Move inputs to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}")

# Save the model and tokenizer
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("Model and tokenizer saved to", SAVE_DIR)

Starting training...


Epoch 1/5: 100%|██████████| 1744/1744 [01:07<00:00, 25.77it/s, loss=0.471]


Epoch 1 completed. Average Loss: 0.4983


Epoch 2/5: 100%|██████████| 1744/1744 [01:08<00:00, 25.63it/s, loss=0.137] 


Epoch 2 completed. Average Loss: 0.3200


...

Epoch 3 completed. Average Loss: 0.2127


Epoch 4/5: 100%|██████████| 1744/1744 [01:08<00:00, 25.63it/s, loss=0.115]  


Epoch 4 completed. Average Loss: 0.1518


Epoch 5/5: 100%|██████████| 1744/1744 [01:08<00:00, 25.63it/s, loss=0.358]  


Epoch 5 completed. Average Loss: 0.1115
Model and tokenizer saved to saved_model_fakenewsnet_roberta_title_smote


In [19]:
# ------------------------------------------------------------------
# Step 8: Evaluate the Model
# ------------------------------------------------------------------
model.eval()
predictions, true_labels = [], []

for batch in tqdm(eval_loader, desc="Evaluating"):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    preds = torch.argmax(outputs.logits, dim=1)
    predictions.extend(preds.cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

acc = accuracy_score(true_labels, predictions)
print("Evaluation Accuracy:", acc)
print(classification_report(true_labels, predictions, target_names=["Real", "Fake"]))

Evaluating: 100%|██████████| 290/290 [00:02<00:00, 110.59it/s]

Evaluation Accuracy: 0.8293103448275863
              precision    recall  f1-score   support

        Real       0.87      0.91      0.89      3489
        Fake       0.68      0.59      0.63      1151

    accuracy                           0.83      4640
   macro avg       0.77      0.75      0.76      4640
weighted avg       0.82      0.83      0.83      4640

